<a href="https://colab.research.google.com/github/jonathan12932/ACES-Cybersecurity-Research/blob/main/1G_TheDefenders_HACS200_DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
import os
import zipfile
import re
from datetime import datetime

import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import plotly.express as px

# Import Logs

Link your google drive to the colab.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Manually

Use the following command in your personal terminal to get new data from honeypots.

```
scp -r -P 48746 student@jump.aces.umd.edu:/home/all_zips/ /path/to/your/dir/
```

Upload the zip file from honeypots to your google drive and proceed to the next section.

## Import Automatically (TBD)

NEW! Get files directly from Honeypots (NOT WORKING AT ALL 🔥🔥🔥)

# Unzip Logs (Optional)

Only for checking the logs manually

DO NOT RUN!

In [ ]:
# The directory containing your zip files
# Uncomment the 2 directories below to run this code without errors
# zip_directory = '/content/drive/MyDrive/all_zips'
# unzip_directory = '/content/drive/MyDrive/all_zips_unzipped'

# Loop through all files in the specified directory
for filename in os.listdir(zip_directory):
    # Check if the file is a zip file
    if filename.endswith('.zip'):
        # Full path to the zip file
        zip_path = os.path.join(zip_directory, filename)

        # Name for the subdirectory to unzip into, based on the zip file name
        output_folder = os.path.join(unzip_directory, filename.replace('.zip', ''))
        os.makedirs(output_folder, exist_ok=True)

        # Unzip the file
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(output_folder)
        print(f"Unzipped {filename} into {output_folder}")


NameError: name 'unzip_directory' is not defined

# Parse Logs

Parse the log files and turn them into more readable csv files containing all necessary data.

---

To parse your own data just replace the "/content/drive/MyDrive/y2s2/HACS200/" part of every dir with the path to your all_zips folder. (see zip_directories)

ex. "/content/drive/MyDrive/all_zips/20241027_171843_zips/ExternalIP_2_zips"

(If you directly dropped all_zips into your google drive without putting it into any folders use this prefix)

In [ ]:
# The directory containing your zip files and end location of csv
zip_directories = ["/content/drive/MyDrive/y2s2/HACS200/all_zips/20241027_171843_zips/ExternalIP_2_zips",
                   "/content/drive/MyDrive/y2s2/HACS200/all_zips/20241028_230012_zips/ExternalIP_2_zips",
                   "/content/drive/MyDrive/y2s2/HACS200/all_zips/ExternalIP_1",
                   "/content/drive/MyDrive/y2s2/HACS200/all_zips/ExternalIP_2",
                   "/content/drive/MyDrive/y2s2/HACS200/all_zips/ExternalIP_3",
                   "/content/drive/MyDrive/y2s2/HACS200/all_zips/ExternalIP_4",
                   "/content/drive/MyDrive/y2s2/HACS200/all_zips/ExternalIP_5"]
csv_directory = '/content/drive/MyDrive/y2s2/HACS200/data_csvs'

# Add more cols for keystrokes later !!!!!!!!!
fields_ui = ["HPt #", "Pwd Str", "Atk #", "Login", "IP In", "ID In", "Logout", "IP Out", "ID Out", "Auth #", "Auth Start", "Auth End", "Auth IP", "Cmd #"]
df_ui = pd.DataFrame(columns=fields_ui)
# Function to parse a directory of zipped log files
def parse(zip_directory, df_ui, fields_ui):
  # Loop through all files in the specified directory
  for filename in os.listdir(zip_directory):
      # Check if the file is a zip file
      if filename.endswith('.zip'):
          # Full path to the zip file
          zip_path = os.path.join(zip_directory, filename)
          # Unzip the zip file as 'arc'
          with zipfile.ZipFile(zip_path, 'r') as arc:
            # Get one attacker instance to add to dataframe
            for file in arc.namelist():
              if file.startswith("logins/honeypot"):
                # Chop off the 'logins/' part of the path to get file's name
                file = file[7:]
                # Initialize a new row for the dataframe
                new_row = ["HPt # NoV", "Pwd Str NoV", "Atk # NoV", "Login NoV",
                          "IP In NoV", "ID In NoV", "Logout NoV", "IP Out NoV",
                          "ID Out NoV", "Auth # NoV", "Auth Start NoV",
                          "Auth End NoV", "Auth IP NoV", "Cmd # NoV"]
                # Set honeypot number – what honeypot attacker came from
                new_row[0] = file[8]
                # Set attacker id number – maybe be duplicates
                new_row[1] = file.split("_")[1]
                # Set password strength – independent variable
                new_row[2] = file.split("_")[-1].split(".")[0]

                # Turn the login and logout files into a readable format
                login = arc.open("logins/"+file)
                logout = arc.open("logouts/"+file)
                login_data = login.read().decode('utf-8').strip()
                logout_data = logout.read().decode('utf-8').strip()
                # Check if both a login and logout was recorded
                if len(login_data) > 0 and len(logout_data) > 0:
                  # Get only first logging attempt (Prevents duplicate logging)
                  login_data = login_data.splitlines()[0]
                  logout_data = logout_data.splitlines()[-1]

                  # Both login and logout IP and ID should be the same
                  # Set all login file associated data
                  new_row[3] = login_data.split(";")[0]
                  new_row[4] = login_data.split(";")[1]
                  new_row[5] = login_data.split(";")[2]
                  # Set all logout file associated data
                  new_row[6] = logout_data.split(";")[0]
                  new_row[7] = logout_data.split(";")[1]
                  new_row[8] = logout_data.split(";")[2]

                  # Get # of cmds
                  if "keystrokes/"+login_data.split(";")[2]+".log" in arc.namelist():
                    kstrs = arc.open("keystrokes/"+login_data.split(";")[2]+".log")
                    kstrs_data = kstrs.read().decode('utf-8').strip()
                    kstrs_data_lines = kstrs_data.splitlines()
                    # Noninteractive and interactive regex patterns
                    #
                    # THIS MAY NOT BE RIGHT!!!
                    # CHANGELOG 1: Added additional cmd check for semicolons
                    # Add check for escaped semicolons?
                    #
                    noninteractive_pattern = r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d+ \[Noninteractive Mode\]'
                    interactive_pattern = r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d+ \[Full Line Parsed\]'
                    cn = 0
                    for line in kstrs_data_lines:
                      # Check if the line matches either a noninteractive or interactive command pattern
                      if re.match(noninteractive_pattern, line) or re.match(interactive_pattern, line):
                        # Check if any trailing or duplicate semicolons
                        cmds = [cmd for cmd in line.split(";") if cmd.strip()]
                        # Register each command separately
                        cn += len(cmds)
                    # Set the number of commands run
                    new_row[13] = cn
                  else:
                    new_row[13] = "NA"

                # If no login/logout exists then set all associated values to "NA"
                else:
                  new_row[3] = "NA"
                  new_row[4] = "NA"
                  new_row[5] = "NA"
                  new_row[6] = "NA"
                  new_row[7] = "NA"
                  new_row[8] = "NA"
                  new_row[13] = "NA"

                if "authentication_attempts/"+file in arc.namelist():
                  # Turn the auth file into a readable format
                  auth = arc.open("authentication_attempts/"+file)
                  auth_data = auth.read().decode('utf-8').strip()
                  auth_data_lines = auth_data.splitlines()
                  # Set the number of auth attempts
                  new_row[9] = len(auth_data_lines)
                  # Check if there were any auth attempts
                  if len(auth_data_lines) > 0:
                    # Set the auth start time
                    new_row[10] = auth_data_lines[0].split(";")[0]
                    # Set the auth end time
                    new_row[11] = auth_data_lines[-1].split(";")[0]
                    # Set the auth IP
                    new_row[12] = auth_data_lines[-1].split(";")[1]
                  # If no auth attempts then set acssociated values to "NA"
                  else:
                    new_row[10] = "NA"
                    new_row[11] = "NA"
                    new_row[12] = "NA"
                else:
                  new_row[10] = "NA"
                  new_row[11] = "NA"
                  new_row[12] = "NA"

                # Update existing dataframe with new data
                df_row = pd.DataFrame([new_row], columns=fields_ui)
                df_ui = pd.concat([df_ui, df_row])
      # break (for testing)
  return df_ui


for zip_directory in zip_directories:
  df_ui = parse(zip_directory, df_ui, fields_ui)
print(len(df_ui))
print(df_ui)

# Save parsed data to drive
df_ui.to_csv(os.path.join(csv_directory, "parsed.csv"), index=False)

24097
   HPt # Pwd Str Atk #                    Login          IP In  \
0      2  Medium   452  2024-10-26 20:03:52.066  152.42.139.72   
0      2    Weak   455  2024-10-26 20:15:54.221  152.42.139.72   
0      2    Weak   453  2024-10-26 20:06:00.813  152.42.250.74   
0      2    None   458  2024-10-26 20:24:09.789  152.42.250.74   
0      2    High   457  2024-10-26 20:21:55.140  152.42.139.72   
..   ...     ...   ...                      ...            ...   
0      5    Weak  3040                       NA             NA   
0      5  Medium  2936                       NA             NA   
0      5  Medium  2957                       NA             NA   
0      5    Weak  2841                       NA             NA   
0      5    None  2862  2024-11-23 22:13:35.433  180.153.91.15   

                      ID In                   Logout         IP Out  \
0   2024_10_26_20_03_52_066  2024-10-26 20:03:52.809  152.42.139.72   
0   2024_10_26_20_15_54_221  2024-10-26 20:15:54.970  152.4

Get a new dataframe for the commands. (SFJA)

In [ ]:
# Get only valid commands into a new dataframe
df_ci = df_ui.copy()
df_ci["Cmd #"] = pd.to_numeric(df_ci["Cmd #"], errors='coerce')
df_ci = df_ci.dropna(subset=["Cmd #"])
df_ci = df_ci.loc[:, ["ID In", "Cmd #"]]
df_ci = df_ci.rename(columns={"ID In": "ID"})
df_ci.drop_duplicates(subset=["ID"], inplace=True)
df_ci.set_index("ID", inplace=True)
print(len(df_ci))
df_ci

17505


,Cmd #
ID,
2024_10_26_20_03_52_066,1.0
2024_10_26_20_15_54_221,1.0
2024_10_26_20_06_00_813,1.0
2024_10_26_20_24_09_789,1.0
2024_10_26_20_21_55_140,1.0
...,...
2024_11_24_05_28_32_041,3.0
2024_11_24_15_59_32_923,3.0
2024_11_24_02_06_21_978,3.0


Parse keystrokes files one more time to get all commands.

In [ ]:
def parse_cmd(zip_directory, df_ci):
  for filename in os.listdir(zip_directory):
      if filename.endswith('.zip'):
          zip_path = os.path.join(zip_directory, filename)
          # Unzip the zip file as 'arc'
          with zipfile.ZipFile(zip_path, 'r') as arc:
            # Get one attacker instance to add to dataframe
            for file in arc.namelist():
              if file.startswith("keystrokes/"):
                log = arc.open(file)
                log_data = log.read().decode('utf-8').strip()
                log_data_lines = log_data.splitlines()
                file = file[11:-4]
                # Noninteractive and interactive regex patterns
                #
                # THIS MAY NOT BE RIGHT!!!
                # CHANGELOG 1: Added additional cmd check for semicolons
                # Add check for escaped semicolons?
                #
                noninteractive_pattern_c = r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d+ \[Noninteractive Mode\] (.*)'
                interactive_pattern_c = r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d+ \[Full Line Parsed\] (.*)'
                cn = 0
                for line in log_data_lines:
                  xim = re.match(noninteractive_pattern_c, line)
                  im = re.match(interactive_pattern_c, line)
                  if xim or im:
                    matched = xim if xim else im
                    # cmds = [cmd for cmd in line.split("]")[1].split(";") if cmd.strip()]
                    cmds = [cmd for cmd in matched.group(1).split(";") if cmd.strip()]
                    # noarg = [cmd.split("\\s+")[0] for cmd in cmds]
                    for cmd in cmds:
                      if cmd not in df_ci.columns:
                        df_ci[cmd] = 0
                        df_ci = df_ci.copy()
                      if file in df_ci.index:
                        df_ci.at[file, cmd] += 1
  return df_ci



if len(df_ci.columns) <= 2:
  for zip_directory in zip_directories:
    df_ci = parse_cmd(zip_directory, df_ci)
print(len(df_ci))
print(df_ci)

# Save parsed commands to drive
df_ci.to_csv(os.path.join(csv_directory, "commands.csv"), index=False)

17505
                         Cmd #  uname -s -v -n -r -m  uname -a  \
ID                                                               
2024_10_26_20_03_52_066    1.0                     1         0   
2024_10_26_20_15_54_221    1.0                     1         0   
2024_10_26_20_06_00_813    1.0                     1         0   
2024_10_26_20_24_09_789    1.0                     1         0   
2024_10_26_20_21_55_140    1.0                     1         0   
...                        ...                   ...       ...   
2024_11_24_05_28_32_041    3.0                     0         0   
2024_11_24_15_59_32_923    3.0                     0         0   
2024_11_24_02_06_21_978    3.0                     0         0   
2024_11_24_02_23_42_266    3.0                     0         0   
2024_11_23_22_13_35_433    3.0                     0         0   

                         sudo hive-passwd set ifjeeisurofmioufiose  \
ID                                                               

# Validate/Clean Data

Perform some preliminary data validity checks before mutating the data.

In [ ]:
# Nothing should be removed from this cleaning step
#
# Tell me if number of rows in df_ui above and df_uic below are not equal (some invalids atm)
#
df_uic = df_ui[df_ui["IP In"] == df_ui["IP Out"]]
# df_uic = df_uic[df_uic["IP In"] == df_uic["Auth IP"]]
df_uic = df_uic[df_uic["ID In"] == df_uic["ID Out"]]
df_uic = df_uic.drop(columns=["IP Out", "ID Out"])
df_uic = df_uic.rename(columns={"IP In": "IP", "ID In": "ID"})
# df_uic.drop_duplicates(inplace=True)
print(len(df_uic))
print(df_uic)

23927
   HPt # Pwd Str Atk #                    Login             IP  \
0      2  Medium   452  2024-10-26 20:03:52.066  152.42.139.72   
0      2    Weak   455  2024-10-26 20:15:54.221  152.42.139.72   
0      2    Weak   453  2024-10-26 20:06:00.813  152.42.250.74   
0      2    None   458  2024-10-26 20:24:09.789  152.42.250.74   
0      2    High   457  2024-10-26 20:21:55.140  152.42.139.72   
..   ...     ...   ...                      ...            ...   
0      5    Weak  3040                       NA             NA   
0      5  Medium  2936                       NA             NA   
0      5  Medium  2957                       NA             NA   
0      5    Weak  2841                       NA             NA   
0      5    None  2862  2024-11-23 22:13:35.433  180.153.91.15   

                         ID                   Logout Auth #  \
0   2024_10_26_20_03_52_066  2024-10-26 20:03:52.809      1   
0   2024_10_26_20_15_54_221  2024-10-26 20:15:54.970      1   
0   2024_10_

Add columns to the dataframe for time logged in, time authenticating, and total time.

In [ ]:
df_uic["Login"] = pd.to_datetime(df_uic["Login"], format="%Y-%m-%d %H:%M:%S.%f", errors="coerce")
df_uic["Logout"] = pd.to_datetime(df_uic["Logout"], format="%Y-%m-%d %H:%M:%S.%f", errors="coerce")
df_uic["Auth Start"] = pd.to_datetime(df_uic["Auth Start"], format="%Y-%m-%d %H:%M:%S.%f", errors="coerce")
df_uic["Auth End"] = pd.to_datetime(df_uic["Auth End"], format="%Y-%m-%d %H:%M:%S.%f", errors="coerce")
df_uic["Time Logged In (s)"] = df_uic.apply(
    lambda row: (row["Logout"] - row["Login"]).total_seconds() if pd.notna(row["Login"]) and pd.notna(row["Logout"]) else "NA",
    axis=1
)
df_uic["Time Auth (s)"] = df_uic.apply(
    lambda row: (row["Auth End"] - row["Auth Start"]).total_seconds() if pd.notna(row["Auth Start"]) and pd.notna(row["Auth End"]) else "NA",
    axis=1
)
df_uic["Total Time (s)"] = df_uic.apply(
    lambda row: (row["Logout"] - row["Auth Start"]).total_seconds() if pd.notna(row["Auth Start"]) and pd.notna(row["Logout"]) else "NA",
    axis=1
)
print(df_uic)

   HPt # Pwd Str Atk #                   Login             IP  \
0      2  Medium   452 2024-10-26 20:03:52.066  152.42.139.72   
0      2    Weak   455 2024-10-26 20:15:54.221  152.42.139.72   
0      2    Weak   453 2024-10-26 20:06:00.813  152.42.250.74   
0      2    None   458 2024-10-26 20:24:09.789  152.42.250.74   
0      2    High   457 2024-10-26 20:21:55.140  152.42.139.72   
..   ...     ...   ...                     ...            ...   
0      5    Weak  3040                     NaT             NA   
0      5  Medium  2936                     NaT             NA   
0      5  Medium  2957                     NaT             NA   
0      5    Weak  2841                     NaT             NA   
0      5    None  2862 2024-11-23 22:13:35.433  180.153.91.15   

                         ID                  Logout Auth #  \
0   2024_10_26_20_03_52_066 2024-10-26 20:03:52.809      1   
0   2024_10_26_20_15_54_221 2024-10-26 20:15:54.970      1   
0   2024_10_26_20_06_00_813 2024-

Add additional cleaning steps below!!!

*   ~~Filter out duplicate IPs~~
*   Filter out testing IP (DONE)
*   Remove all entries with no login (DONE)
*   Get number of cd commands (No cd so far)
*   Mark bot activity (uname only)
*   etc.

In [ ]:
# TODO
# Filter out the testing IP
df_uid = df_uic[df_uic["IP"] != "10.172.195.4"]
# Remove invalid entries
# df_uif = df_uif[df_uic["Total Time (s)"] != "NA"]
# Remove people who were not able to log in
# df_uif = df_uif[df_uic["Time Logged In (s)"] != "NA"]
print(len(df_uid))
df_uid.to_csv(os.path.join(csv_directory, "cleaned.csv"), index=False)

23919


# Testing

Used to testing contents of dataframes.